In [1]:
import pandas as pd
import datetime, time
from datetime import timedelta
import requests, os, zipfile
from zipfile import ZipFile
#declaring the data path
data_path = 'F://stock market/Quant/Jupiter/Deliverables/'
Daily_Data_Path = 'F://stock market/Quant/Jupiter/Deliverables/dwnld/'

today = datetime.date.today() 					### 2021-03-08  
# if(today.strftime('%a') == 'Sat' or today.strftime('%a') == 'Sun'):
#     break
# else:
print(today)
month = today.strftime("%m")			### 03
# day = '06'
day = today.strftime("%d")				### 08
MON = today.strftime("%b").upper()		### MAR
year = today.year 						### 2021
year_in_2_digit = today.strftime("%y")	### 21
    
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:84.0) Gecko/20100101 Firefox/84.0','referer':'https://www1.nseindia.com/'}

#making dynamic urls
NSE_Bhavcopy_url = f'https://www1.nseindia.com/content/historical/EQUITIES/{year}/{MON}/cm{day}{MON}{year}bhav.csv.zip'
NSE_Deliverables_url = f'https://www1.nseindia.com/archives/equities/mto/MTO_{day}{month}{year}.DAT'

def clean_Daily_Data_Folder():
    if not os.path.exists(Daily_Data_Path):
        os.mkdir(Daily_Data_Path)
    for file in os.listdir(Daily_Data_Path):
        os.chdir(Daily_Data_Path)
        try:	
            os.remove(file)
        except:
            shutil.rmtree(Daily_Data_Path+file)
            pass
    print("Starting download...""\n")
    
def NSE_Bhavcopy():
    r = requests.get(NSE_Bhavcopy_url, headers=headers)
    if r.status_code == 200:
        with open(Daily_Data_Path + 'NSE_Bhavcopy.zip','wb') as f:
            f.write(r.content)
            print('NSE Bhavcopy Download Completed')
    else:
        print('NSE Bhavcopy Download Failed')

def NSE_Deliverables():
    r = requests.get(NSE_Deliverables_url, headers=headers)
    if r.status_code == 200:
        with open(Daily_Data_Path + 'NSE_Deliverables_Data.csv','wb') as f:
            f.write(r.content)
            print('NSE Deliverables Data Download Complete')
    else:
        print('NSE Deliverables Data Download')

def BSE_Bhavcopy():
    r = requests.get(BSE_Bhavcopy_url,headers=headers)
    if r.status_code == 200:
        with open(Daily_Data_Path + 'BSE_Bhavcopy.zip','wb') as f:
            f.write(r.content)
            print("BSE Bhavcopy download Complete")
    else:
        print('BSE Bhavcopy download failed')

def BSE_Deliverables():
    r = requests.get(BSE_Deliverables_url,headers=headers)
    if r.status_code == 200:
        with open(Daily_Data_Path + 'BSE_Deliverables.zip','wb') as f:
            f.write(r.content)
            print("BSE Deliverables data download Complete")
    else:
        print('BSE Deliverables data download failed')            


#extracting and deleting zip file
def extract():
    print('Extracting Downloaded Files')
    os.chdir(Daily_Data_Path)
    for file in os.listdir(Daily_Data_Path):
        if file.endswith('.zip'):
            with ZipFile(file, 'r') as zipobj:
                zipobj.extractall()
            os.remove(file)
    print('Extraction Completed Successfully')
    print()

try:
    clean_Daily_Data_Folder()
    NSE_Bhavcopy()
    NSE_Deliverables()
#     BSE_Bhavcopy()
#     BSE_Deliverables()
except Exception as e:
    print(e)
    
if len(os.listdir(Daily_Data_Path)) == 2:
    print()
    extract()
#     Zipfiles(data_path)
else:
    print('Data not available for download')

#reading values from NSE Bhavcopy
df = pd.read_csv(f'{Daily_Data_Path}cm{day}{MON}{year}bhav.csv')
df = df[df['SERIES'] == 'EQ']
df = df[['SYMBOL', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'PREVCLOSE', 'TOTTRDVAL', 'TIMESTAMP', 'TOTALTRADES']]
df =df.reset_index(drop=True).set_index('SYMBOL')
# df
da = pd.read_csv(f'{Daily_Data_Path}NSE_Deliverables_Data.csv', names=["Record Type","Sr No","Name of Security","Series","Volume","Delivery Volume","Delivery Percentage"])
da = da[da['Series'] == 'EQ'].drop(['Record Type','Sr No'],axis = 1).reset_index(drop=True).set_index('Name of Security')

dc = pd.DataFrame.join(df, da).reset_index()
# dc.loc[i, 'TIMESTAMP']

for i in range (0, len(df)):
    Ticker = dc.loc[i, 'SYMBOL']
    Ticker_columns = ['Date', 'Open', 'High', 'Low', 'Close','Previous Close', 'Total Traded Value', 'Total Trades',  'Volume', 'Delivered Volume', 'Delivery Percentage']
    try:
        Ticker_df = pd.read_csv(f'{data_path}/data/{Ticker}.csv')
    except:
        Ticker_df = pd.DataFrame(columns = Ticker_columns)
    length = len(Ticker_df)
    if Ticker_df.loc[length-1, 'Date'] == today:
        continue
    else:
        Ticker_df = Ticker_df.append(pd.Series([today,
                                                      dc.loc[i, 'OPEN'],
                                                      dc.loc[i, 'HIGH'],
                                                      dc.loc[i, 'LOW'],
                                                      dc.loc[i, 'CLOSE'],
                                                      dc.loc[i, 'PREVCLOSE'], 
                                                      dc.loc[i, 'TOTTRDVAL'],
                                                      dc.loc[i, 'TOTALTRADES'],
                                                      dc.loc[i, 'Volume'],
                                                      dc.loc[i, 'Delivery Volume'], 
                                                      dc.loc[i, 'Delivery Percentage']],
                                                    index = Ticker_columns),
                                           ignore_index = True)
        Ticker_df.to_csv(f'{data_path}/data/{Ticker}.csv', index= False)

2021-07-22
Starting download...

NSE Bhavcopy Download Completed
NSE Deliverables Data Download Complete

Extracting Downloaded Files
Extraction Completed Successfully



KeyError: -1

In [2]:
import pandas as pd
import datetime, time
from datetime import timedelta
import requests, os, zipfile
from zipfile import ZipFile
#declaring the data path
data_path = 'F://stock market/Quant/Jupiter/Deliverables/'
Daily_Data_Path = 'F://stock market/Quant/Jupiter/Deliverables/dwnld/'

for i in range(0,2):
    #parsing the date 
    today = datetime.date.today() - timedelta(days = 1-i) 					### 2021-03-08  
    if(today.strftime('%a') == 'Sat' or today.strftime('%a') == 'Sun'):
        continue
    else:
        print(today)
    month = today.strftime("%m")			### 03
    # day = '06'
    day = today.strftime("%d")				### 08
    MON = today.strftime("%b").upper()		### MAR
    year = today.year 						### 2021
    year_in_2_digit = today.strftime("%y")	### 21
    
    headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:84.0) Gecko/20100101 Firefox/84.0','referer':'https://www1.nseindia.com/'}

    #making dynamic urls
    NSE_Bhavcopy_url = f'https://www1.nseindia.com/content/historical/EQUITIES/{year}/{MON}/cm{day}{MON}{year}bhav.csv.zip'
    NSE_Deliverables_url = f'https://www1.nseindia.com/archives/equities/mto/MTO_{day}{month}{year}.DAT'

    def clean_Daily_Data_Folder():
        if not os.path.exists(Daily_Data_Path):
            os.mkdir(Daily_Data_Path)
        for file in os.listdir(Daily_Data_Path):
            os.chdir(Daily_Data_Path)
            try:	
                os.remove(file)
            except:
                shutil.rmtree(Daily_Data_Path+file)
                pass
        print("Starting download...""\n")
    
    def NSE_Bhavcopy():
        r = requests.get(NSE_Bhavcopy_url, headers=headers)
        if r.status_code == 200:
            with open(Daily_Data_Path + 'NSE_Bhavcopy.zip','wb') as f:
                f.write(r.content)
                print('NSE Bhavcopy Download Completed')
        else:
            print('NSE Bhavcopy Download Failed')

    def NSE_Deliverables():
        r = requests.get(NSE_Deliverables_url, headers=headers)
        if r.status_code == 200:
            with open(Daily_Data_Path + 'NSE_Deliverables_Data.csv','wb') as f:
                f.write(r.content)
                print('NSE Deliverables Data Download Complete')
        else:
            print('NSE Deliverables Data Download')

    def BSE_Bhavcopy():
        r = requests.get(BSE_Bhavcopy_url,headers=headers)
        if r.status_code == 200:
            with open(Daily_Data_Path + 'BSE_Bhavcopy.zip','wb') as f:
                f.write(r.content)
                print("BSE Bhavcopy download Complete")
        else:
            print('BSE Bhavcopy download failed')

    def BSE_Deliverables():
        r = requests.get(BSE_Deliverables_url,headers=headers)
        if r.status_code == 200:
            with open(Daily_Data_Path + 'BSE_Deliverables.zip','wb') as f:
                f.write(r.content)
                print("BSE Deliverables data download Complete")
        else:
            print('BSE Deliverables data download failed')            


    #extracting and deleting zip file
    def extract():
        print('Extracting Downloaded Files')
        os.chdir(Daily_Data_Path)
        for file in os.listdir(Daily_Data_Path):
            if file.endswith('.zip'):
                with ZipFile(file, 'r') as zipobj:
                    zipobj.extractall()
                os.remove(file)
        print('Extraction Completed Successfully')
        print()

#     def Zipfiles(path):
#         os.chdir(path)
#         files = os.listdir(path)
#         file_name = f'{day}{MON}{year_in_2_digit}.zip'
#         if len(files) == 0:
#             print('Directory is empty, Nothing to Unzip!!')
#         else:
#             with zipfile.ZipFile(file_name, 'w', compression = zipfile.ZIP_DEFLATED) as zf:
#                 for file in files:
#                     zf.write(file)
#             print('Packed and ready for Backup')

    try:
        clean_Daily_Data_Folder()
        NSE_Bhavcopy()
        NSE_Deliverables()
#     BSE_Bhavcopy()
#     BSE_Deliverables()
    except Exception as e:
        print(e)
    
    if len(os.listdir(Daily_Data_Path)) == 2:
        print()
        extract()
#     Zipfiles(data_path)
    else:
        print('Data not available for download')

#reading values from NSE Bhavcopy
    try:
        df = pd.read_csv(f'{Daily_Data_Path}cm{day}{MON}{year}bhav.csv')
    except:
        continue
    df = df[df['SERIES'] == 'EQ']
    df = df[['SYMBOL', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'PREVCLOSE', 'TOTTRDVAL', 'TIMESTAMP', 'TOTALTRADES']]
    df =df.reset_index(drop=True).set_index('SYMBOL')
# df
    da = pd.read_csv(f'{Daily_Data_Path}NSE_Deliverables_Data.csv', names=["Record Type","Sr No","Name of Security","Series","Volume","Delivery Volume","Delivery Percentage"])
    da = da[da['Series'] == 'EQ'].drop(['Record Type','Sr No'],axis = 1).reset_index(drop=True).set_index('Name of Security')

    dc = pd.DataFrame.join(df, da).reset_index()
# dc.loc[i, 'TIMESTAMP']

    for i in range (0, len(df)):
        Ticker = dc.loc[i, 'SYMBOL']
        Ticker_columns = ['Date', 'Open', 'High', 'Low', 'Close','Previous Close', 'Total Traded Value', 'Total Trades',  'Volume', 'Delivered Volume', 'Delivery Percentage']
        try:
            Ticker_df = pd.read_csv(f'{data_path}/data/{Ticker}.csv')
        except:
            Ticker_df = pd.DataFrame(columns = Ticker_columns)
        Ticker_df = Ticker_df.append(pd.Series([today,
                                                      dc.loc[i, 'OPEN'],
                                                      dc.loc[i, 'HIGH'],
                                                      dc.loc[i, 'LOW'],
                                                      dc.loc[i, 'CLOSE'],
                                                      dc.loc[i, 'PREVCLOSE'], 
                                                      dc.loc[i, 'TOTTRDVAL'],
                                                      dc.loc[i, 'TOTALTRADES'],
                                                      dc.loc[i, 'Volume'],
                                                      dc.loc[i, 'Delivery Volume'], 
                                                      dc.loc[i, 'Delivery Percentage']],
                                                    index = Ticker_columns),
                                           ignore_index = True)
        Ticker_df.to_csv(f'{data_path}/data/{Ticker}.csv', index= False)

2021-07-19
Starting download...

NSE Bhavcopy Download Completed
NSE Deliverables Data Download Complete

Extracting Downloaded Files
Extraction Completed Successfully

2021-07-20
Starting download...

NSE Bhavcopy Download Completed
NSE Deliverables Data Download Complete

Extracting Downloaded Files
Extraction Completed Successfully



In [14]:
today = datetime.date.today() - timedelta(days = 38)
if(today.strftime('%a') == 'Sat' or today.strftime('%a') == 'Sun'):
     print('yes')

yes


In [ ]:
df = pd.read_csv(f'{data_path}SCBSEALL{day}{month}.txt', sep = '|')
df

In [ ]:
# df = pd.read_csv(f'{data_path}cm{day}{MON}{year}bhav.csv')
# df
import pandas as pd
data_path = 'F://stock market/Quant/Jupiter/Deliverables/'
df = pd.read_csv(f'{data_path}cm06JUL2021bhav.csv')
df = df[df['SERIES'] == 'EQ']
df = df[['SYMBOL', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'PREVCLOSE', 'TOTTRDVAL', 'TIMESTAMP', 'TOTALTRADES']]
df =df.reset_index(drop=True).set_index('SYMBOL')
df

da = pd.read_csv(f'{data_path}NSE_Deliverables_Data.csv', names=["Record Type","Sr No","Name of Security","Series","Volume","Delivery Volume","Delivery Percentage"])
da = da[da['Series'] == 'EQ'].drop(['Record Type','Sr No'],axis = 1).reset_index(drop=True).set_index('Name of Security')


dc = pd.DataFrame.join(df, da)
dc.reset_index()

fot i in range (0, len(df)):
    Ticker = dc.loc[i, 'SYMBOL']
    f'{Ticker}_columns' = ['Date', 'Open', 'High', 'Low', 'Close',
                           'Previous Close', 'Total Traded Value', 'Total Trades',  'Volume',
                           'Delivered Volume','Delivery Percentage']
    f'{Ticker}_df' = pd.DataFrame(columns = f'{Tciker}_Columns')
    f'{Ticker}_df' = f'{Ticker}_df'.append(pd.Series([dc.loc[f'{Tikcer}', 'TIMESTAMP'],
                                                      dc.loc[f'{Tikcer}', 'OPEN'],
                                                      dc.loc[f'{Tikcer}', 'HIGH'],
                                                      dc.loc[f'{Tikcer}', 'LOW'],
                                                      dc.loc[f'{Tikcer}', 'CLOSE'],
                                                      dc.loc[f'{Tikcer}', 'PREVCLOSE'], 
                                                      dc.loc[f'{Tikcer}', 'TOTTRDVAL'],
                                                      dc.loc[f'{Tikcer}', 'TOTALTRADES'],
                                                      dc.loc[f'{Tikcer}', 'Volume'],
                                                      dc.loc[f'{Tikcer}', 'Delivery Volume'], 
                                                      dc.loc[f'{Tikcer}', 'Delivery Percentage']],
                                                    index = f'{Ticker}_Columns'),
                                           ignore_index = True)
    f'{Ticker}_df'.to_csv(f'{Ticker}.csv')

In [24]:
# import pandas as pd
# import datetime, time
# from datetime import timedelta
# import requests, os, zipfile
# from zipfile import ZipFile
# #declaring the data path
# data_path = 'F://stock market/Quant/Jupiter/Deliverables/'

# #parsing the date 
# today = datetime.date.today() - timedelta(days = 36) 					### 2021-03-08  
# print(today)
# month = today.strftime("%m")			### 03
# # day = '06'
# day = today.strftime("%d")				### 08
# MON = today.strftime("%b").upper()		### MAR
# year = today.year 						### 2021
# year_in_2_digit = today.strftime("%y")	### 21


# headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:84.0) Gecko/20100101 Firefox/84.0','referer':'https://www1.nseindia.com/'}

# #making dynamic urls
# NSE_Bhavcopy_url = f'https://www1.nseindia.com/content/historical/EQUITIES/{year}/{MON}/cm{day}{MON}{year}bhav.csv.zip'
# NSE_Deliverables_url = f'https://www1.nseindia.com/archives/equities/mto/MTO_{day}{month}{year}.DAT'

# def NSE_Bhavcopy():
#     r = requests.get(NSE_Bhavcopy_url, headers=headers)
#     if r.status_code == 200:
#         with open(data_path + 'NSE_Bhavcopy.zip','wb') as f:
#             f.write(r.content)
#             print('NSE Bhavcopy Download Completed')
#     else:
#         print('NSE Bhavcopy Download Failed')

# def NSE_Deliverables():
#     r = requests.get(NSE_Deliverables_url, headers=headers)
#     if r.status_code == 200:
#         with open(data_path + 'NSE_Deliverables_Data.csv','wb') as f:
#             f.write(r.content)
#             print('NSE Deliverables Data Download Complete')
#     else:
#         print('NSE Deliverables Data Download')

# def BSE_Bhavcopy():
#     r = requests.get(BSE_Bhavcopy_url,headers=headers)
#     if r.status_code == 200:
#         with open(Daily_Data_Path + 'BSE_Bhavcopy.zip','wb') as f:
#             f.write(r.content)
#             print("BSE Bhavcopy download Complete")
#     else:
#         print('BSE Bhavcopy download failed')

# def BSE_Deliverables():
#     r = requests.get(BSE_Deliverables_url,headers=headers)
#     if r.status_code == 200:
#         with open(Daily_Data_Path + 'BSE_Deliverables.zip','wb') as f:
#             f.write(r.content)
#             print("BSE Deliverables data download Complete")
#     else:
#         print('BSE Deliverables data download failed')            


# #extracting and deleting zip file
# def extract():
#     print('Extracting Downloaded Files')
#     os.chdir(data_path)
#     for file in os.listdir(data_path):
#         if file.endswith('.zip'):
#             with ZipFile(file, 'r') as zipobj:
#                 zipobj.extractall()
#             os.remove(file)
#     print('Extraction Completed Successfully')
#     print()

# def Zipfiles(path):
#     os.chdir(path)
#     files = os.listdir(path)
#     file_name = f'{day}{MON}{year_in_2_digit}.zip'
#     if len(files) == 0:
#         print('Directory is empty, Nothing to Unzip!!')
#     else:
#         with zipfile.ZipFile(file_name, 'w', compression = zipfile.ZIP_DEFLATED) as zf:
#             for file in files:
#                 zf.write(file)
#         print('Packed and ready for Backup')

# try:
#     NSE_Bhavcopy()
#     NSE_Deliverables()
# #     BSE_Bhavcopy()
# #     BSE_Deliverables()
# except Exception as e:
#     print(e)
    
# if len(os.listdir(data_path)) == 5:
#     print()
#     extract()
# #     Zipfiles(data_path)
# else:
#     print('Data not available for download')

# #reading values from NSE Bhavcopy
# df = pd.read_csv(f'{data_path}cm{day}{MON}{year}bhav.csv')
# df = df[df['SERIES'] == 'EQ']
# df = df[['SYMBOL', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'PREVCLOSE', 'TOTTRDVAL', 'TIMESTAMP', 'TOTALTRADES']]
# df =df.reset_index(drop=True).set_index('SYMBOL')
# # df
# da = pd.read_csv(f'{data_path}NSE_Deliverables_Data.csv', names=["Record Type","Sr No","Name of Security","Series","Volume","Delivery Volume","Delivery Percentage"])
# da = da[da['Series'] == 'EQ'].drop(['Record Type','Sr No'],axis = 1).reset_index(drop=True).set_index('Name of Security')

# dc = pd.DataFrame.join(df, da).reset_index()
# # dc.loc[i, 'TIMESTAMP']

# for i in range (0, len(df)):
#     Ticker = dc.loc[i, 'SYMBOL']
#     Ticker_columns = ['Date', 'Open', 'High', 'Low', 'Close','Previous Close', 'Total Traded Value', 'Total Trades',  'Volume', 'Delivered Volume', 'Delivery Percentage']
#     try:
#         Ticker_df = pd.read_csv(f'{data_path}/data/{Ticker}.csv')
#     except:
#         Ticker_df = pd.DataFrame(columns = Ticker_columns)
#     Ticker_df = Ticker_df.append(pd.Series([dc.loc[i, 'TIMESTAMP'],
#                                                       dc.loc[i, 'OPEN'],
#                                                       dc.loc[i, 'HIGH'],
#                                                       dc.loc[i, 'LOW'],
#                                                       dc.loc[i, 'CLOSE'],
#                                                       dc.loc[i, 'PREVCLOSE'], 
#                                                       dc.loc[i, 'TOTTRDVAL'],
#                                                       dc.loc[i, 'TOTALTRADES'],
#                                                       dc.loc[i, 'Volume'],
#                                                       dc.loc[i, 'Delivery Volume'], 
#                                                       dc.loc[i, 'Delivery Percentage']],
#                                                     index = Ticker_columns),
#                                            ignore_index = True)
#     Ticker_df.to_csv(f'{data_path}/data/{Ticker}.csv', index= False)

In [ ]:
da = pd.read_csv(f'{data_path}NSE_Deliverables_Data.csv', names=["Record Type","Sr No","Name of Security","Series","Volume","Delivery Volume","Delivery Percentage"])

# da = da.set_index('Sr No')
# dc = da.loc['20MICRONS', 'Quantity Traded']


da


In [ ]:
dr = pd.read_csv(f'{data_path}EQ_ISINCODE_060721.csv').reset_index(drop=True)
dr = dr[dr['SC_TYPE'] == 'Q']
dr.set_index('SC_NAME')
# dr.loc['HDFC', 'OPEN']
dr

In [ ]:
import datetime
day = datetime.date.today()
day